In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid', font_scale=1)
sns.set_palette('Set2', n_colors=10)
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)


import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', 50)

In [2]:
tr = pd.read_csv('../data/acc_train.csv')
te = pd.read_csv('../data/acc_test.csv')
sp_acc = pd.read_csv('../data/sp_accident.csv')
df = pd.concat([tr,te])

In [3]:
df.head(3)

,my_id,사고일시,사고지역,사망자수,중상자수,경상자수,부상신고자수,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자차종,가해운전자성별,가해운전자연령,가해운전자상해정도,피해운전자차종,피해운전자성별,피해운전자연령,피해운전자상해정도,음주운전
0,A000002,2020년 4월 16일 05시,영등포구 여의도동,0,1,0,0,차대사람 - 횡단중,안전운전불이행,포장 - 건조,맑음,단일로 - 기타,승용,남,59.0,상해없음,보행자,남,71.0,중상,0.0
1,A000003,2020년 3월 3일 22시,성북구 동소문동3가,0,0,1,0,차대차 - 추돌,안전거리미확보,포장 - 건조,맑음,단일로 - 기타,승용,남,66.0,상해없음,승용,남,40.0,경상,0.0
2,A000004,2019년 5월 17일 18시,구로구 가리봉동,0,0,1,0,차대차 - 측면충돌,신호위반,포장 - 건조,맑음,교차로 - 교차로안,승용,남,42.0,상해없음,개인형이동수단(PM),남,63.0,경상,0.0


In [4]:
sp_acc.head()

,my_id,유형
0,A052133,무면허
1,A074188,무면허
2,A070801,무면허
3,A017799,무면허
4,A038642,무면허


In [5]:
sp_acc['유형'].value_counts()

택시사고         11082
렌터카사고         4062
무단횡단사고        2727
무면허           1129
어린이보호구역사고      554
터널사고           229
Name: 유형, dtype: int64

In [6]:
temp = sp_acc.groupby(['my_id','유형'])['유형'].count().unstack().reset_index()
temp

유형,my_id,렌터카사고,무단횡단사고,무면허,어린이보호구역사고,택시사고,터널사고
0,A000001,NaN,NaN,NaN,NaN,1.0,NaN
1,A000002,1.0,1.0,NaN,NaN,NaN,NaN
2,A000003,NaN,NaN,NaN,NaN,1.0,NaN
3,A000005,NaN,NaN,NaN,NaN,1.0,NaN
4,A000007,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...
18853,A074474,NaN,NaN,NaN,NaN,1.0,NaN
18854,A074476,NaN,1.0,NaN,NaN,NaN,NaN
18855,A074478,NaN,NaN,NaN,NaN,1.0,NaN
18856,A074481,NaN,NaN,NaN,NaN,1.0,NaN


In [7]:
df = pd.merge(left = df , right = temp, how = "left", on = "my_id")
df.head(3)

,my_id,사고일시,사고지역,사망자수,중상자수,경상자수,부상신고자수,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자차종,가해운전자성별,가해운전자연령,가해운전자상해정도,피해운전자차종,피해운전자성별,피해운전자연령,피해운전자상해정도,음주운전,렌터카사고,무단횡단사고,무면허,어린이보호구역사고,택시사고,터널사고
0,A000002,2020년 4월 16일 05시,영등포구 여의도동,0,1,0,0,차대사람 - 횡단중,안전운전불이행,포장 - 건조,맑음,단일로 - 기타,승용,남,59.0,상해없음,보행자,남,71.0,중상,0.0,1.0,1.0,NaN,NaN,NaN,NaN
1,A000003,2020년 3월 3일 22시,성북구 동소문동3가,0,0,1,0,차대차 - 추돌,안전거리미확보,포장 - 건조,맑음,단일로 - 기타,승용,남,66.0,상해없음,승용,남,40.0,경상,0.0,NaN,NaN,NaN,NaN,1.0,NaN
2,A000004,2019년 5월 17일 18시,구로구 가리봉동,0,0,1,0,차대차 - 측면충돌,신호위반,포장 - 건조,맑음,교차로 - 교차로안,승용,남,42.0,상해없음,개인형이동수단(PM),남,63.0,경상,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
values = {"렌터카사고": 0, "무단횡단사고": 0, "무면허": 0, "어린이보호구역사고": 0,"택시사고": 0,"터널사고": 0}
df = df.fillna(values)

- 사고일시

In [9]:
df['사고일시'] = pd.to_datetime(df['사고일시'], format="%Y년 %m월 %d일 %H시", errors='coerce')

In [10]:
#분기
df['분기'] = df['사고일시'].dt.quarter

In [11]:
#월
df['월'] = df['사고일시'].dt.month

In [12]:
#요일
df['요일'] = df['사고일시'].dt.day_of_week

In [13]:
#시간
df['시간'] = df['사고일시'].dt.hour

In [14]:
# df[df['음주운전'] == 1.0]['시간'].value_counts()

In [15]:
#음주운전 발생시간 20시~08시:1 / 나머지시간:0
df['음주운전_시간'] = df['시간'].apply(lambda x: 0
                            if (x in np.arange(9.0,19.0))
                            else 1)

- 사고지역 

In [16]:
df['사고지역'].value_counts()

강남구 역삼동     1621
관악구 신림동     1496
서초구 서초동     1386
노원구 상계동     1342
강서구 화곡동     1267
            ... 
서대문구 봉원동       2
종로구 권농동        2
종로구 송현동        1
중구 수하동         1
중구 회현동3가       1
Name: 사고지역, Length: 465, dtype: int64

In [17]:
df['사고지역'] = df['사고지역'].str.split(' ').str[0]

In [18]:
# df['사고지역'].value_counts()

In [19]:
# df[df['음주운전'] == 1][['음주운전', '사고지역']].value_counts()

In [20]:
# df[df['음주운전'] == 0][['음주운전', '사고지역']].value_counts()

In [21]:
df['사고지역'] = df['사고지역'].apply(lambda x: df['사고지역'].value_counts().idxmax()
                            if (x =='류')
                            else x)

In [22]:
# df['사고지역'].value_counts()

- 사고유형

In [23]:
df['사고유형'].value_counts()

차대차 - 측면충돌           24627
차대차 - 기타             15940
차대차 - 추돌             11194
차대사람 - 기타             7486
차대사람 - 횡단중            6106
차대차 - 정면충돌            1933
차대사람 - 차도통행중          1711
차량단독 - 기타             1565
차대사람 - 보도통행중          1226
차대사람 - 길가장자리구역통행중     1038
차대차 - 후진중충돌            964
차량단독 - 공작물충돌           442
차량단독 - 전도전복 - 전도       221
차량단독 - 전도전복 - 전복        12
차량단독 - 도로외이탈 - 추락       10
차량단독 - 도로외이탈 - 기타        6
차량단독 - 주/정차차량 충돌         4
Name: 사고유형, dtype: int64

In [24]:
# df[df['음주운전'] == 1][['음주운전', '사고유형']].value_counts()

In [25]:
# df[df['음주운전'] == 0][['음주운전', '사고유형']].value_counts()

In [26]:
df['사고유형_대범주'] = df['사고유형'].str.split(' - ').str[0]

In [27]:
df['사고유형_대범주'].value_counts()

차대차     54658
차대사람    17567
차량단독     2260
Name: 사고유형_대범주, dtype: int64

In [28]:
df['단독사고'] = df['사고유형_대범주'].apply(lambda x: 1
                            if (x =='차량단독')
                            else 0)

In [29]:
df['보행자사고'] = df['사고유형_대범주'].apply(lambda x: 1
                            if (x =='차대사람')
                            else 0)

- 법규위반

In [30]:
df['법규위반'].value_counts()

안전운전불이행      41379
안전거리미확보      10369
신호위반          9001
기타            2828
보행자보호의무위반     2693
교차로운행방법위반     2647
중앙선침범         2027
직진우회전진행방해     1475
차로위반          1349
불법유턴           513
과속             204
Name: 법규위반, dtype: int64

In [31]:
# df[df['음주운전'] == 1][['음주운전', '법규위반']].value_counts()

In [32]:
# df[df['음주운전'] == 0][['음주운전', '법규위반']].value_counts()

- 노면상태

In [33]:
df['노면상태'].value_counts()

포장 - 건조        67200
포장 - 젖음/습기      6033
포장 - 기타         1087
포장 - 서리/결빙        60
포장 - 적설           39
비포장 - 젖음/습기       27
비포장 - 건조          24
비포장 - 기타           8
포장 - 해빙            3
포장 - 침수            3
비포장 - 해빙           1
Name: 노면상태, dtype: int64

In [34]:
df['노면상태'] = df['노면상태'].apply(lambda x: 1
                            if (x =='포장')
                            else 0)

- 도로형태 대범주

In [35]:
df['도로형태'].value_counts()

단일로 - 기타           36061
교차로 - 교차로안         18715
교차로 - 교차로부근        10739
기타 - 기타             4056
교차로 - 교차로횡단보도내      2788
단일로 - 지하차도(도로)내      734
단일로 - 교량             588
단일로 - 고가도로위          306
주차장 - 주차장            246
단일로 - 터널             229
미분류 - 미분류             23
Name: 도로형태, dtype: int64

In [36]:
# df[df['음주운전'] == 1][['음주운전', '도로형태']].value_counts()

In [37]:
# df[df['음주운전'] == 0][['음주운전', '도로형태']].value_counts()

In [38]:
df['도로형태_대범주'] = df['도로형태'].str.split(' - ').str[0]

In [39]:
df['도로형태_대범주'].value_counts()

단일로    37918
교차로    32242
기타      4056
주차장      246
미분류       23
Name: 도로형태_대범주, dtype: int64

- 가해운전자성별

In [40]:
df['가해운전자성별'] = df['가해운전자성별'].apply(lambda x: 1
                            if (x =='남성')
                            else 0)

- 가해운전자차종

In [41]:
df['가해운전자차종'] = df['가해운전자차종'].apply(lambda x: 1
                            if (x =='승용')
                            else 0)

- 피해운전자차종

In [42]:
df['피해운전자성별'] = df['피해운전자성별'].apply(lambda x: 1
                            if (x =='남성')
                            else 0)

- 피해운전자차종

In [43]:
df['피해운전자차종'] = df['피해운전자차종'].apply(lambda x: 1
                            if (x =='승용')
                            else 0)

- 가해운전자상해정도

In [44]:
# nan, 상해없음, 경상, 기타불명, 부상신고 - 0
# 중상, 사망 - 1

In [45]:
df['가해운전자상해정도'].unique()

array(['상해없음', '부상신고', '경상', nan, '중상', '사망'], dtype=object)

In [46]:
df['가해운전자상해정도'] = df['가해운전자상해정도'].apply(lambda x: 1
                            if (x in ['중상','사망'])
                            else 0)

- 피해운전자상해정도

In [47]:
df['피해운전자상해정도'].unique()

array(['중상', '경상', '부상신고', '상해없음', nan, '기타불명', '사망'], dtype=object)

In [48]:
df['피해운전자상해정도'] = df['피해운전자상해정도'].apply(lambda x: 1
                            if (x in ['중상','사망'])
                            else 0)

- 필요없는 컬럼 drop

In [49]:
# df = df.drop(['사고일시','사고유형','노면상태','기상상태','도로형태','가해운전자상해정도','피해운전자차종','피해운전자성별','피해운전자연령','피해운전자상해정도'], axis=1)

In [50]:
# df = df.drop(['사고일시','기상상태','가해운전자상해정도','피해운전자차종','피해운전자성별','피해운전자연령','피해운전자상해정도'], axis=1)

In [51]:
# df = df.drop(['사고일시','사고지역','사고유형','기상상태','노면상태','도로형태','가해운전자상해정도','피해운전자차종','피해운전자성별','피해운전자연령','피해운전자상해정도'], axis=1)

In [52]:
df = df.drop(['사고일시','기상상태'], axis=1)

In [53]:
df.dtypes

my_id         object
사고지역          object
사망자수           int64
중상자수           int64
경상자수           int64
부상신고자수         int64
사고유형          object
법규위반          object
노면상태           int64
도로형태          object
가해운전자차종        int64
가해운전자성별        int64
가해운전자연령      float64
가해운전자상해정도      int64
피해운전자차종        int64
피해운전자성별        int64
피해운전자연령      float64
피해운전자상해정도      int64
음주운전         float64
렌터카사고        float64
무단횡단사고       float64
무면허          float64
어린이보호구역사고    float64
택시사고         float64
터널사고         float64
분기             int64
월              int64
요일             int64
시간             int64
음주운전_시간        int64
사고유형_대범주      object
단독사고           int64
보행자사고          int64
도로형태_대범주      object
dtype: object

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74485 entries, 0 to 74484
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   my_id      74485 non-null  object 
 1   사고지역       74485 non-null  object 
 2   사망자수       74485 non-null  int64  
 3   중상자수       74485 non-null  int64  
 4   경상자수       74485 non-null  int64  
 5   부상신고자수     74485 non-null  int64  
 6   사고유형       74485 non-null  object 
 7   법규위반       74485 non-null  object 
 8   노면상태       74485 non-null  int64  
 9   도로형태       74485 non-null  object 
 10  가해운전자차종    74485 non-null  int64  
 11  가해운전자성별    74485 non-null  int64  
 12  가해운전자연령    72727 non-null  float64
 13  가해운전자상해정도  74485 non-null  int64  
 14  피해운전자차종    74485 non-null  int64  
 15  피해운전자성별    74485 non-null  int64  
 16  피해운전자연령    72049 non-null  float64
 17  피해운전자상해정도  74485 non-null  int64  
 18  음주운전       59589 non-null  float64
 19  렌터카사고      74485 non-null  float64
 20  무단횡단사고

In [55]:
df = df.set_index('my_id')

In [56]:
num_columns = df.select_dtypes(['number']).drop(columns='음주운전').columns.tolist()
print(num_columns)

['사망자수', '중상자수', '경상자수', '부상신고자수', '노면상태', '가해운전자차종', '가해운전자성별', '가해운전자연령', '가해운전자상해정도', '피해운전자차종', '피해운전자성별', '피해운전자연령', '피해운전자상해정도', '렌터카사고', '무단횡단사고', '무면허', '어린이보호구역사고', '택시사고', '터널사고', '분기', '월', '요일', '시간', '음주운전_시간', '단독사고', '보행자사고']


In [57]:
cat_columns = df.select_dtypes(['object']).columns.tolist()
print(cat_columns)

['사고지역', '사고유형', '법규위반', '도로형태', '사고유형_대범주', '도로형태_대범주']


In [58]:
df.head()

,사고지역,사망자수,중상자수,경상자수,부상신고자수,사고유형,법규위반,노면상태,도로형태,가해운전자차종,가해운전자성별,가해운전자연령,가해운전자상해정도,피해운전자차종,피해운전자성별,피해운전자연령,피해운전자상해정도,음주운전,렌터카사고,무단횡단사고,무면허,어린이보호구역사고,택시사고,터널사고,분기,월,요일,시간,음주운전_시간,사고유형_대범주,단독사고,보행자사고,도로형태_대범주
my_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A000002,영등포구,0,1,0,0,차대사람 - 횡단중,안전운전불이행,0,단일로 - 기타,1,0,59.0,0,0,0,71.0,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2,4,3,5,1,차대사람,0,1,단일로
A000003,성북구,0,0,1,0,차대차 - 추돌,안전거리미확보,0,단일로 - 기타,1,0,66.0,0,1,0,40.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,3,1,22,1,차대차,0,0,단일로
A000004,구로구,0,0,1,0,차대차 - 측면충돌,신호위반,0,교차로 - 교차로안,1,0,42.0,0,0,0,63.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5,4,18,0,차대차,0,0,교차로
A000005,구로구,0,1,0,0,차대차 - 측면충돌,신호위반,0,교차로 - 교차로안,1,0,57.0,0,1,0,57.0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,11,0,7,1,차대차,0,0,교차로
A000006,마포구,0,0,1,0,차대차 - 측면충돌,안전거리미확보,0,교차로 - 교차로부근,0,0,52.0,0,1,0,38.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5,1,14,0,차대차,0,0,교차로


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74485 entries, A000002 to A020850
Data columns (total 33 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   사고지역       74485 non-null  object 
 1   사망자수       74485 non-null  int64  
 2   중상자수       74485 non-null  int64  
 3   경상자수       74485 non-null  int64  
 4   부상신고자수     74485 non-null  int64  
 5   사고유형       74485 non-null  object 
 6   법규위반       74485 non-null  object 
 7   노면상태       74485 non-null  int64  
 8   도로형태       74485 non-null  object 
 9   가해운전자차종    74485 non-null  int64  
 10  가해운전자성별    74485 non-null  int64  
 11  가해운전자연령    72727 non-null  float64
 12  가해운전자상해정도  74485 non-null  int64  
 13  피해운전자차종    74485 non-null  int64  
 14  피해운전자성별    74485 non-null  int64  
 15  피해운전자연령    72049 non-null  float64
 16  피해운전자상해정도  74485 non-null  int64  
 17  음주운전       59589 non-null  float64
 18  렌터카사고      74485 non-null  float64
 19  무단횡단사고     74485 non-null  float64
 20  무면허

In [60]:
week = pd.get_dummies(df['요일'],prefix ='요일')
df = df.join(week)

In [61]:
col = ['요일']
df = df.drop(columns=col)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74485 entries, A000002 to A020850
Data columns (total 39 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   사고지역       74485 non-null  object 
 1   사망자수       74485 non-null  int64  
 2   중상자수       74485 non-null  int64  
 3   경상자수       74485 non-null  int64  
 4   부상신고자수     74485 non-null  int64  
 5   사고유형       74485 non-null  object 
 6   법규위반       74485 non-null  object 
 7   노면상태       74485 non-null  int64  
 8   도로형태       74485 non-null  object 
 9   가해운전자차종    74485 non-null  int64  
 10  가해운전자성별    74485 non-null  int64  
 11  가해운전자연령    72727 non-null  float64
 12  가해운전자상해정도  74485 non-null  int64  
 13  피해운전자차종    74485 non-null  int64  
 14  피해운전자성별    74485 non-null  int64  
 15  피해운전자연령    72049 non-null  float64
 16  피해운전자상해정도  74485 non-null  int64  
 17  음주운전       59589 non-null  float64
 18  렌터카사고      74485 non-null  float64
 19  무단횡단사고     74485 non-null  float64
 20  무면허

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74485 entries, A000002 to A020850
Data columns (total 39 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   사고지역       74485 non-null  object 
 1   사망자수       74485 non-null  int64  
 2   중상자수       74485 non-null  int64  
 3   경상자수       74485 non-null  int64  
 4   부상신고자수     74485 non-null  int64  
 5   사고유형       74485 non-null  object 
 6   법규위반       74485 non-null  object 
 7   노면상태       74485 non-null  int64  
 8   도로형태       74485 non-null  object 
 9   가해운전자차종    74485 non-null  int64  
 10  가해운전자성별    74485 non-null  int64  
 11  가해운전자연령    72727 non-null  float64
 12  가해운전자상해정도  74485 non-null  int64  
 13  피해운전자차종    74485 non-null  int64  
 14  피해운전자성별    74485 non-null  int64  
 15  피해운전자연령    72049 non-null  float64
 16  피해운전자상해정도  74485 non-null  int64  
 17  음주운전       59589 non-null  float64
 18  렌터카사고      74485 non-null  float64
 19  무단횡단사고     74485 non-null  float64
 20  무면허

In [63]:
df.to_pickle('../pkl/0416.pkl')